In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf

import pickle
import os, sys, time
import shap

In [6]:
def create_data(data_path, normalization = False, scaler = None):
    dataFrame = pd.read_excel(data_path, sheet_name='Data')
    string_to_number_set = []
    for col in dataFrame:
        unique_strings = dataFrame.loc[dataFrame[col].apply(lambda x: isinstance(x, str)), col].unique()
        string_to_number = {s: i + 1 for i, s in enumerate(unique_strings)}
        dataFrame[col] = dataFrame[col].replace(string_to_number)
        string_to_number_set.append(string_to_number)
    all_info = dataFrame.iloc[:, 4:32]
    features = all_info.iloc[:, :16]
    targets = all_info.iloc[:, 23:28]
    features_header = features.columns.values
    
    

    # Find columns with NaN
    slump_data = pd.concat([features, targets.iloc[:, 0]], axis=1).dropna(axis=0, how='any').values
    comp_str_data = pd.concat([features, targets.iloc[:, 1]], axis=1).dropna(axis=0, how='any').values
    split_str_data = pd.concat([features, targets.iloc[:, 2]], axis=1).dropna(axis=0, how='any').values
    chloride_data = pd.concat([features, targets.iloc[:, 3]], axis=1).dropna(axis=0, how='any').values
    carbon_data = pd.concat([features, targets.iloc[:, 4]], axis=1).dropna(axis=0, how='any').values
    
    X = [
        slump_data[:, :-1],
        comp_str_data[:,:-1],
        split_str_data[:,:-1],
        chloride_data[:,:-1],
        carbon_data[:,:-1]
    ]

    Y = [
        slump_data[:, -1:],
        comp_str_data[:, -1:],
        split_str_data[:, -1:],
        chloride_data[:, -1:],
        carbon_data[:, -1:]
    ]
    

In [7]:
# Define the dataPreparation function
def dataPreparation(X, Y, test_size, random_state, D_contra=True):
    assert len(X) == len(Y)
    T = len(Y)  # Number of tasks
    O = [y.shape[1] for y in Y]  # Number of outputs for every task
    D = [x.shape[1] for x in X]
    if D_contra:
        assert len(set(D)) == 1
        D = list(set(D))[0]  # Dimension of feature vector

    X_train = []; Y_train = []; X_test = []; Y_test = []
    for i in range(T):
        x_train, x_test, y_train, y_test = train_test_split(X[i], Y[i], test_size=test_size, random_state=random_state)
        X_train.append(x_train)
        Y_train.append(y_train)
        X_test.append(x_test)
        Y_test.append(y_test)

    return X_train, Y_train, X_test, Y_test, T, O, D

In [8]:
# Define the model architecture using the Keras functional API
def build_model(input_shape, output_shape):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu')(inputs)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(output_shape)(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
# Load and preprocess data
data_dir = 'C:/Users/leeno/Desktop/Dataspell/4102/'
data_name = 'allData.xls'
data_path = os.path.join(data_dir, data_name)
X_all, y_all, scaler, features_header = create_data(data_path, normalization=True)

C:\Users\leeno\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[  0.   0. 450.   0.   0.   0.   0. 180.   4.  20. 990. 800.   1.  20.
 100.   3.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.